**Outline:**
- Build model and loss function
- Train model
- Observe valdidate
- Test

**To do:**
- Hyperparameter tuning
    + lr
    + layer1, layer2
    + betas

**Modification**
- Weight initialization with xavier uniform
- Adam optimization
- LR decay

In [1]:
import importlib
import argparse
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle
import time

In [2]:
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler
from torch.autograd import Variable
from torchvision import transforms

In [3]:
sys.path.insert(0, './../utils/')
from utils import *
from models import *

In [4]:
#Define parser
#name = 'bpi_2012'
#name = 'bpi_2013'
name = 'small_log'
#name = 'large_log'

parser = {
    'train': True,
    'test': True,
    'model_class': 'AE',
    'model_name': '',
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'nan_pct': 0.5,
    'input_dir': '../input/{}/'.format(name),
    'batch_size' : 16,
    'epochs' : 200,
    'no_cuda' : False,
    'seed' : 7,
    'layer1': 300,
    'layer2': 100,
    'lr': 0.0005,
    'betas': (0.9, 0.999),   
    'lr_decay': 0.90,
}

args = argparse.Namespace(**parser)
args.output_dir = './output/{0}_{1}_{2}/'.format(name, args.nan_pct, args.model_class)

In [5]:
if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

In [6]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [7]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [8]:
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

In [9]:
preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_full_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'rb') as f:
    min_max_storage = pickle.load(f)
    complete_matrix_w_normalized_time = pickle.load(f)
    missing_matrix_w_normalized_time = pickle.load(f)
    avai_matrix = pickle.load(f)
    nan_matrix = pickle.load(f)
    pad_matrix = pickle.load(f)
    cols_w_time = pickle.load(f)
    cols_w_normalized_time = pickle.load(f)

In [10]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

# Load data

In [11]:
complete_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(complete_matrix_w_normalized_time, 
                                                                            batch_size=args.batch_size, shuffle=False, 
                                                                            num_workers=2)
missing_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(missing_matrix_w_normalized_time, 
                                                                           batch_size=args.batch_size, shuffle=False, 
                                                                           num_workers=2)
avai_matrix_trainLoader = torch.utils.data.DataLoader(avai_matrix, 
                                                      batch_size=args.batch_size, shuffle=False, 
                                                      num_workers=2)

In [12]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

In [13]:
missing_true_test = normalized_missing_df
complete_true_test = normalized_complete_df

In [14]:
nan_time_index, nan_activity_index = getnanindex(missing_true_test)

In [15]:
row_num = missing_true_test.shape[0]

In [16]:
complete_matrix_w_normalized_time.shape

(13087, 175, 37)

In [17]:
35*8

280

# Build model

## Define model

In [18]:
if args.model_class == 'AE':
    model = AE(complete_matrix_w_normalized_time.shape, args.layer1, args.layer2)
        
if args.cuda:
    model.cuda()

## Define loss

In [19]:
# Define loss

def loss_function(recon_x, x, avai_mask):
    #MSE = F.mse_loss(recon_x*avai_mask, x*avai_mask, size_average=False)
    BCE = F.binary_cross_entropy(recon_x, x, weight=avai_mask, size_average=False) 
    return BCE

In [20]:
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=args.betas)

In [21]:
#Adjust learning rate per epoch: http://pytorch.org/docs/master/optim.html?highlight=adam#torch.optim.Adam

# Method 1:
lambda1 = lambda epoch: args.lr_decay ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

# Method 2:
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)

## Utils

In [22]:
def save_model(model, epoch, score):
    model_file = os.path.join(args.output_dir, 'model_{}_epoch{}_score{:.4f}.pth'.format(args.model_class, epoch, score))
    torch.save(model.state_dict(), model_file)

In [23]:
def load_model(model, model_name):
    model_file = os.path.join(args.output_dir, model_name)
    assert os.path.isfile(model_file), 'Error: no model found!'
    model_state = torch.load(model_file)
    model.load_state_dict(model_state)

# Train model

In [24]:
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (m_data, c_data, avai_mask) in enumerate(zip(missing_matrix_w_normalized_time_trainLoader, 
                                                                complete_matrix_w_normalized_time_trainLoader,
                                                                avai_matrix_trainLoader)):
        c_data = Variable(c_data.float())
        m_data = Variable(m_data.float())
        avai_mask = Variable(avai_mask.float())

        if args.cuda:
            c_data = c_data.cuda()
            m_data = m_data.cuda()
            avai_mask = avai_mask.cuda()

            
        optimizer.zero_grad()
        
        recon_data = model(m_data)
        
        loss = loss_function(recon_data, c_data, avai_mask)
        
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        
    return train_loss / len(complete_matrix_w_normalized_time_trainLoader.dataset)

In [25]:
if args.train:
    for epoch in range(1, args.epochs + 1):
        init = time.time()
        
        #method 1 scheduler
        scheduler.step()
        train_loss = train(epoch, model, optimizer)
        
        end = time.time()
        print('====> Epoch {} | End time: {:.4f} ms | Train loss: {:.4f}'.
              format(epoch, (end-init)*1000, train_loss))
else:
    load_model(model, args.model_name)

====> Epoch 1 | End time: 2927.8162 ms | Train loss: 41.2679
====> Epoch 2 | End time: 2809.6933 ms | Train loss: 25.1099
====> Epoch 3 | End time: 2809.2673 ms | Train loss: 20.7302
====> Epoch 4 | End time: 2920.8090 ms | Train loss: 18.0386
====> Epoch 5 | End time: 2883.6315 ms | Train loss: 16.0032
====> Epoch 6 | End time: 2835.0167 ms | Train loss: 14.4409
====> Epoch 7 | End time: 2860.0199 ms | Train loss: 13.1798
====> Epoch 8 | End time: 2867.3451 ms | Train loss: 12.1145
====> Epoch 9 | End time: 2827.0733 ms | Train loss: 11.2195
====> Epoch 10 | End time: 2859.0631 ms | Train loss: 10.4843
====> Epoch 11 | End time: 2853.2701 ms | Train loss: 9.8688
====> Epoch 12 | End time: 2872.5953 ms | Train loss: 9.3499
====> Epoch 13 | End time: 2827.5106 ms | Train loss: 8.9095
====> Epoch 14 | End time: 2870.7354 ms | Train loss: 8.5345
====> Epoch 15 | End time: 2859.7374 ms | Train loss: 8.2115
====> Epoch 16 | End time: 2903.3062 ms | Train loss: 7.9322
====> Epoch 17 | End ti

# Predict and evaluate

In [26]:
if args.test:
    m_test = missing_matrix_w_normalized_time
    m_test = Variable(torch.Tensor(m_test).float())
    
    if args.cuda:
        m_test = m_test.cuda()
    
    print('Predicting...')
    recon_test = model(m_test)
    
    print('\n')
    print('Converting to dataframe...')
    recon_df_w_normalized_time = convert2df(recon_test, pad_matrix, cols_w_normalized_time, row_num)
    
    print('Transforming Normalized Time to Time...')
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage)
    
    print('Getting submission...')
    submission_df = getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp)
    submission = fixTime(submission_df)
    
    print('Testing...')
    mae_time, rmse_time, acc = evaluation(submission, nan_time_index, nan_activity_index, show=True)
    print('\n')
    
    print('Saving submission...')
    submission_df.to_csv(args.output_dir+'submission.csv', index=False)
    print('Done!')

Predicting...


Converting to dataframe...
Transforming Normalized Time to Time...
Getting submission...


/home/pllab/miniconda2/envs/pydata/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Testing...
Number of missing Time: 131056
Mean Absolute Error: 1.1615 day(s)
Root Mean Squared Error: 3.0603 day(s)
Number of missing Activity: 131144
Accuracy: 77.68%


Saving submission...
Done!


In [27]:
submission_df.shape

(262200, 7)

In [28]:
submission.head(10)

,CaseID,TrueActivity,PredictedActivity,TrueTime,PredictedTime,TrueCompleteTimestamp,PredictedCompleteTimestamp
0,1,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,0.000,0.000000,2011-10-01 07:38:44.546,2011-10-01 07:38:44.546
1,1,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,0.334,0.334000,2011-10-01 07:38:44.880,2011-10-01 07:38:44.880
2,1,A_PREACCEPTED-COMPLETE,A_PREACCEPTED-COMPLETE,53.360,53.360000,2011-10-01 07:39:37.906,2011-10-01 07:39:37.906
3,1,W_Completeren aanvraag-SCHEDULE,W_Completeren aanvraag-SCHEDULE,54.329,54.329000,2011-10-01 07:39:38.875,2011-10-01 07:39:38.875
4,1,W_Completeren aanvraag-START,W_Completeren aanvraag-START,39481.891,39481.891000,2011-10-01 18:36:46.437,2011-10-01 18:36:46.437
5,1,A_ACCEPTED-COMPLETE,A_ACCEPTED-COMPLETE,39838.762,39838.762000,2011-10-01 18:42:43.308,2011-10-01 18:42:43.308
6,1,O_SELECTED-COMPLETE,O_SELECTED-COMPLETE,39984.697,39838.762000,2011-10-01 18:45:09.243,2011-10-01 18:42:43.308
7,1,A_FINALIZED-COMPLETE,A_FINALIZED-COMPLETE,39984.697,39838.762000,2011-10-01 18:45:09.243,2011-10-01 18:42:43.308
8,1,O_CREATED-COMPLETE,O_CREATED-COMPLETE,39986.651,39986.651000,2011-10-01 18:45:11.197,2011-10-01 18:45:11.197
9,1,O_SENT-COMPLETE,O_SENT-COMPLETE,39986.834,43217.449792,2011-10-01 18:45:11.380,2011-10-01 19:39:01.995792


In [29]:
missing_true_test.head(10)

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,1,A_SUBMITTED-COMPLETE,2011-10-01 07:38:44.546,0.000,0.000000e+00
1,1,NaN,2011-10-01 07:38:44.880,0.334,3.113567e-07
2,1,A_PREACCEPTED-COMPLETE,2011-10-01 07:39:37.906,53.360,4.974249e-05
3,1,W_Completeren aanvraag-SCHEDULE,2011-10-01 07:39:38.875,54.329,5.064580e-05
4,1,W_Completeren aanvraag-START,2011-10-01 18:36:46.437,39481.891,3.680524e-02
5,1,A_ACCEPTED-COMPLETE,2011-10-01 18:42:43.308,39838.762,3.713792e-02
6,1,O_SELECTED-COMPLETE,NaN,NaN,NaN
7,1,NaN,NaN,NaN,NaN
8,1,NaN,2011-10-01 18:45:11.197,39986.651,3.727578e-02
9,1,NaN,NaN,NaN,NaN
